# TOPIC

In [4]:
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.errors import TopicAlreadyExistsError, UnknownTopicOrPartitionError
import time

BROKER = "100.68.89.127:9092"
TOPIC = "pf_bigdata"

def topic_exists(admin: KafkaAdminClient, topic_name: str) -> bool:
    """Retorna True si el topic existe en el broker."""
    return topic_name in admin.list_topics()

def wait_until_deleted(admin: KafkaAdminClient, topic_name: str, timeout: int = 20) -> bool:
    """Espera a que Kafka elimine completamente un topic antes de recrearlo."""
    start = time.time()
    while time.time() - start < timeout:
        if not topic_exists(admin, topic_name):
            return True
        time.sleep(1)
    return False

def create_topic(topic_name: str, partitions: int = 1, replication: int = 1):
    """Elimina un topic si existe y lo recrea de forma segura."""
    admin = KafkaAdminClient(bootstrap_servers=BROKER, client_id="topic-manager")

    try:
        admin.delete_topics([topic_name])
    except UnknownTopicOrPartitionError:
        pass

    wait_until_deleted(admin, topic_name)

    new_topic = NewTopic(
        name=topic_name,
        num_partitions=partitions,
        replication_factor=replication
    )

    try:
        admin.create_topics([new_topic])
        print(f"Topic '{topic_name}' recreado correctamente.")
    except TopicAlreadyExistsError:
        print(f"No se pudo crear el topic '{topic_name}': aún existe.")
    finally:
        admin.close()

# Uso
create_topic(TOPIC, partitions=3, replication=1)


Topic 'pf_bigdata' recreado correctamente.
